In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('train.csv')
data.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [13]:
X = data.drop('label',axis=1)
y = data['label']

0    1
Name: label, dtype: int64

In [14]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X.loc[:5000,:],y[:5001])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [48]:
y_ = clf.predict(X.loc[0:500,:])

In [49]:
y_

array([1, 0, 1, 4, 0, 0, 7, 3, 5, 3, 8, 9, 1, 3, 3, 1, 2, 0, 7, 5, 8, 6, 2,
       0, 2, 3, 6, 9, 9, 7, 8, 9, 4, 9, 2, 1, 3, 1, 1, 4, 9, 1, 4, 4, 2, 6,
       3, 7, 7, 4, 7, 5, 1, 9, 0, 2, 2, 3, 9, 1, 1, 1, 5, 0, 6, 3, 4, 8, 1,
       0, 3, 9, 6, 2, 6, 4, 7, 1, 4, 1, 5, 4, 8, 9, 2, 9, 9, 8, 9, 6, 3, 6,
       4, 6, 2, 9, 1, 2, 0, 5, 9, 2, 7, 7, 2, 8, 8, 5, 0, 6, 0, 0, 2, 9, 0,
       4, 7, 7, 1, 5, 7, 9, 4, 6, 1, 5, 7, 6, 5, 0, 4, 8, 7, 6, 1, 8, 7, 3,
       7, 3, 1, 0, 3, 4, 5, 4, 0, 5, 4, 0, 3, 5, 1, 0, 8, 3, 7, 0, 9, 6, 6,
       9, 5, 4, 6, 9, 3, 5, 4, 2, 4, 8, 7, 7, 5, 8, 8, 8, 2, 6, 9, 3, 1, 0,
       4, 1, 5, 9, 0, 6, 2, 1, 3, 0, 6, 0, 0, 8, 3, 2, 0, 0, 6, 0, 0, 4, 7,
       2, 7, 1, 9, 9, 3, 9, 8, 4, 6, 6, 5, 3, 8, 1, 8, 7, 1, 3, 7, 6, 3, 6,
       3, 6, 3, 2, 3, 2, 2, 7, 9, 2, 3, 2, 7, 5, 5, 8, 8, 2, 0, 1, 4, 0, 6,
       3, 7, 1, 1, 1, 4, 7, 0, 2, 9, 2, 0, 5, 6, 0, 8, 9, 6, 2, 0, 0, 7, 2,
       0, 4, 2, 0, 9, 1, 6, 9, 3, 0, 0, 2, 0, 6, 8, 4, 0, 7, 2, 1, 9, 5, 2,
       4, 8,

0       1
1       0
2       1
3       4
4       0
5       0
6       7
7       3
8       5
9       3
10      8
11      9
12      1
13      3
14      3
15      1
16      2
17      0
18      7
19      5
20      8
21      6
22      2
23      0
24      2
25      3
26      6
27      9
28      9
29      7
       ..
4970    7
4971    7
4972    2
4973    8
4974    3
4975    8
4976    3
4977    6
4978    7
4979    7
4980    3
4981    6
4982    7
4983    0
4984    9
4985    1
4986    3
4987    0
4988    2
4989    8
4990    7
4991    1
4992    4
4993    0
4994    7
4995    2
4996    5
4997    8
4998    7
4999    9
Name: label, dtype: int64